In [ ]:
##UPLOAD

In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account


In [2]:
# These first two values will be different on your machine. 
service_path = ""
service_file = 'wedge-project-key.json' # change this to your authentication information  

gbq_proj_id = 'wedge-project-403915'  

# And this should stay the same. 
private_key = service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(private_key)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
for item in client.list_datasets() : 
    print(item.full_dataset_id)

wedge-project-403915:Transactions
wedge-project-403915:Transactions_test


In [3]:
dataset_id = 'wedge-project-403915'


In [23]:
# Your provided setup for connecting to BigQuery
service_path = ""  # Specify your service path if needed
service_file = 'wedge-project-key.json'  # Change this to your authentication information
gbq_proj_id = 'wedge-project-403915'  # Your project ID
private_key = service_path + service_file

# Establishing connection to BigQuery
credentials = service_account.Credentials.from_service_account_file(private_key)
client = bigquery.Client(credentials=credentials, project=gbq_proj_id)

# Directory containing CSV files
folder_path = "C:/Users/james/OneDrive/Documents/ADA/Wedge_Project/Wedge-Project/wedge-data/wedge-clean-files"  # Replace with the path to your folder containing CSV files

# BigQuery dataset and table names
dataset_name = "Transactions_test"  # The name of your dataset

# Schema definition
schema = [
    bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),  # FLOAT to accommodate potential decimal values
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),  # FLOAT for the same reason
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),  # FLOAT for the same reason
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),  # Mixed types, so STRING is safer
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),  # FLOAT to accommodate potential decimal values
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),  # Contains NaNs
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),  # Contains NaNs and changed to FLOAT
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),  # Contains NaNs and changed to FLOAT
    bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),  # Contains NaNs and changed to FLOAT
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),  # Contains NaNs and changed to FLOAT
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),  # Changed to FLOAT
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),  # Changed to FLOAT
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),  # Changed to FLOAT
    bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),  # Contains NaNs and changed to FLOAT
    bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE")   # Changed to FLOAT
]

# Loop through all CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)

        # Create a table name based on the CSV file name
        table_name = file_name.replace('.csv', '')

        # Full table ID
        table_id = f"{gbq_proj_id}.{dataset_name}.{table_name}"

        # Create a new table
        table = bigquery.Table(table_id, schema=schema)
        table = client.create_table(table, exists_ok=True)  # Set exists_ok=True to avoid errors if table already exists

        # Configuration for loading data
        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV,
            skip_leading_rows=1,  # Adjust based on your CSV file
            schema=schema  # Use the specified schema
        )

        # Load data from CSV file to the newly created BigQuery table
        with open(file_path, "rb") as source_file:
            load_job = client.load_table_from_file(source_file, table_id, job_config=job_config)

        # Waits for the job to complete
        load_job.result()

        print(f"Loaded {file_path} into {table_id}")

print("All files have been loaded to BigQuery.")

KeyboardInterrupt: 

In [22]:
load_job.errors

[{'reason': 'invalid',
  'message': 'Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 100; errors: 100. Please look into the errors[] collection for more details.'},
 {'reason': 'invalid',
  'message': 'Error while reading data, error message: CSV processing encountered too many errors, giving up. Rows: 100; errors: 100; max bad: 0; error percent: 0'},
 {'reason': 'invalid',
  'message': 'Error while reading data, error message: Unable to parse; line_number: 2 byte_offset_to_start_of_line: 461 column_index: 16 column_name: "altPrice" column_type: DOUBLE value: "\\\\N"'},
 {'reason': 'invalid',
  'message': 'Error while reading data, error message: Unable to parse; line_number: 3 byte_offset_to_start_of_line: 613 column_index: 16 column_name: "altPrice" column_type: DOUBLE value: "\\\\N"'},
 {'reason': 'invalid',
  'message': 'Error while reading data, error message: Unable to parse; line_number: 4 byte_offset_to_start_of_line: 797 column_